# Scraping Funding Data
As the Funding per school was not available in a clean format, we decided to write a scraping script

## 0. Import Statements

In [57]:
import pandas as pd
import urllib
import numpy as np
from bs4 import BeautifulSoup

### 0.1 Using DBN to identify schools
We filter for High schools only

In [61]:
df = pd.read_csv('school_codes.csv')
df_narrow = df[df['school_type'] == 'High School']
df_narrow.head(5)

,dbn,report_type,school_type,school
12,01M292,HS,High School,Orchard Collegiate Academy
20,01M448,HS,High School,University Neighborhood High School
22,01M450,HS,High School,East Side Community School
24,01M509,HS,High School,Marta Valle High School
27,01M539,HS,High School,"New Explorations into Science, Technology and ..."


In [58]:
df.head(5)

,dbn,report_type,school_type,school
0,01M015,EMS,Elementary,P.S. 015 Roberto Clemente
1,01M019,EMS,Elementary,P.S. 019 Asher Levy
2,01M020,EMS,Elementary,P.S. 020 Anna Silver
3,01M034,EMS,K-8,P.S. 034 Franklin D. Roosevelt
4,01M063,EMS,Elementary,The STAR Academy - P.S.63


In [62]:
school_codes = list(df_narrow.dbn)
school_codes_clean = [code[2:6] for code in school_codes]

## 1. Scrape Script
In the HTML file we identified the two spans we want to scrape, these are saved in the variables span_id1 and span_id2. They correspond to the Funding given in AY 2012-2013, and AY 2013-2014, respectively.

In [63]:
base_url = "https://www.nycenet.edu/publicapps/Offices/FSF/Report.aspx?year=2014&schoolcode="
span_id1 = "ContentPlaceHolder1_ctl00_lblL01_R01_a"
span_id2 = "ContentPlaceHolder1_ctl00_lblL01_R07_g"

In [64]:
sample_url = base_url + school_codes_clean[0]

### 1.1 Scrape Function and its Output
We output any unsuccessful attemps at scraping. These are uncessful attemps as the schools did not receive Funding from NYC government.

In [65]:
data = []

for i in range(len(school_codes_clean)):
    url = base_url + school_codes_clean[i]
    
    try:
        f = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(f)
        temp = []

        budget_ay12_13 = str(soup.find("span", {"id": span_id1})).split('>')[1].split('<')[0][1:]
        budget_ay12_13 = budget_ay12_13.replace(',', "")

        budget_ay13_14 = str(soup.find("span", {"id": span_id2})).split('>')[1].split('<')[0][1:]
        budget_ay13_14 = budget_ay13_14.replace(',', "")

        temp.append(school_codes_clean[i])
        temp.append(budget_ay12_13)
        temp.append(budget_ay13_14)
        
        data.append(temp)
    
    except Exception as e:
        print(e)
        print(school_codes_clean[i])

/Users/noel/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/noel/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


list index out of range
M047
list index out of range
M280
list index out of range
M282
list index out of range
M211
list index out of range
K765
list index out of range
K764
list index out of range
Q315
list index out of range
Q314
list index out of range
Q334
list index out of range
Q313
list index out of range
R028
list index out of range
K356
list index out of range
K357
list index out of range
K358
list index out of range
K359
list index out of range
K473
list index out of range
K508
list index out of range
K517
list index out of range
K538
list index out of range
K608
list index out of range
K626
list index out of range
K652
list index out of range
K693
list index out of range
K707
list index out of range
K712
list index out of range
K730
list index out of range
K733
list index out of range
K738
list index out of range
K739
list index out of range
K744
list index out of range
K757
list index out of range
K775
list index out of range
K803
list index out of range
K892
list index out

In [66]:
print('Done')

Done


In [79]:
fundingDF = pd.DataFrame.from_records(data)

In [80]:
fundingDF = fundingDF.rename(index=str, columns={0: "DBN", 1: "AY12-13", 2: "AY13-14"})

In [81]:
fundingDF.head(5)

,DBN,AY12-13,AY13-14
0,M292,2357841,2276174
1,M448,2190592,1967439
2,M450,3745068,3700670
3,M509,2198272,2302771
4,M539,6808136,7034224


In [82]:
fundingDF.to_csv("funding_2012_2014.csv")